In [51]:
import sys
import cv2
import numpy as np
from pydensecrf import densecrf as dcrf
from pydensecrf.utils import unary_from_labels, create_pairwise_bilateral, create_pairwise_gaussian,unary_from_softmax

In [53]:
print("IMAGE and ANNO are inputs and OUTPUT is where the result should be written.")
print("If there's at least one single full-black pixel in ANNO, black is assumed to mean unknown.")
    
fn_im = './truck_ref.png'
fn_anno = './truck_inverted.jpeg'
fn_output = './output.png'

img = cv2.imread(fn_im)

# Convert the annotation's RGB color to a single 32-bit integer color 0xBBGGRR
anno_rgb = cv2.cvtColor(cv2.imread(fn_anno), cv2.COLOR_BGR2GRAY)
img = cv2.pyrDown(img)
img = cv2.pyrDown(img)
img = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)
print(anno_rgb.shape, img.shape)

# anno_rgb = cv2.resize(anno_rgb, (640, 480)).astype(np.uint32)
# anno_lbl = anno_rgb[:,:,0] + (anno_rgb[:,:,1] << 8) + (anno_rgb[:,:,2] << 16)
anno_lbl = anno_rgb[:,:]

# img = cv2.resize(img, (anno_rgb.shape[1], anno_rgb.shape[0]))
# Convert the 32bit integer color to 1, 2, ... labels.
# Note that all-black, i.e. the value 0 for background will stay 0.
colors, labels = np.unique(anno_lbl, return_inverse=True)
# print(colors, labels)
# But remove the all-0 black, that won't exist in the MAP!
HAS_UNKNOWN_LABEL = 255 in colors
# HAS_UNKNOWN_LABEL = False
if HAS_UNKNOWN_LABEL:
    print("Found a full-black pixel in annotation image, assuming it means 'unknown' label, and will thus not be present in the output!")
    print("If 0 is an actual label for you, consider writing your own code, or simply giving your labels only non-zero values.")
    colors = colors[:-1]
#else:
#    print("No single full-black pixel found in annotation image. Assuming there's no 'unknown' label!")

# And create a mapping back from the labels to 32bit integer colors.
colorize = np.empty((len(colors), 1), np.uint8)
colorize[:,0] = (colors & 0x0000FF)
# colorize[:,1] = (colors & 0x00FF00) >> 8
# colorize[:,2] = (colors & 0xFF0000) >> 16
print(colorize.shape)

# Compute the number of classes in the label image.
# We subtract one because the number shouldn't include the value 0 which stands
# for "unknown" or "unsure".
n_labels = len(set(labels.flat)) - int(HAS_UNKNOWN_LABEL)
print(n_labels, " labels", (" plus \"unknown\" 0: " if HAS_UNKNOWN_LABEL else ""), set(labels.flat))

###########################
### Setup the CRF model ###
###########################
print(n_labels)
d = dcrf.DenseCRF2D(img.shape[1], img.shape[0], n_labels)

# get unary potentials (neg log probability)
U = unary_from_labels(labels, n_labels, gt_prob=0.9, zero_unsure=HAS_UNKNOWN_LABEL)
print(labels.shape, n_labels)
print(U)
d.setUnaryEnergy(U)

# # This adds the color-independent term, features are the locations only.
# d.addPairwiseGaussian(sxy=(3, 3), compat=13, kernel=dcrf.DIAG_KERNEL,
#                       normalization=dcrf.NORMALIZE_SYMMETRIC)

# This adds the color-dependent term, i.e. features are (x,y,r,g,b).
d.addPairwiseBilateral(sxy=(3, 3), srgb=(20, 20, 20), rgbim=img,
                       compat=np.array([2, 37*0.15]),
                       kernel=dcrf.DIAG_KERNEL,
                       normalization=dcrf.NORMALIZE_SYMMETRIC)

####################################
### Do inference and compute MAP ###
####################################

# Run five inference steps.
Q = d.inference(12)

# Find out the most probable class for each pixel.
MAP = np.argmax(Q, axis=0)

# Convert the MAP (labels) back to the corresponding colors and save the image.
# Note that there is no "unknown" here anymore, no matter what we had at first.
MAP = colorize[MAP,:]
# print(MAP.shape, img.shape)
cv2.imwrite(fn_output, 255 - MAP.reshape(img.shape[:2]))

# Just randomly manually run inference iterations
# Q, tmp1, tmp2 = d.startInference()
# for i in range(15):
#     print("KL-divergence at {}: {}".format(i, d.klDivergence(Q)))
#     d.stepInference(Q, tmp1, tmp2)

IMAGE and ANNO are inputs and OUTPUT is where the result should be written.
If there's at least one single full-black pixel in ANNO, black is assumed to mean unknown.
(270, 480) (270, 480, 3)
Found a full-black pixel in annotation image, assuming it means 'unknown' label, and will thus not be present in the output!
If 0 is an actual label for you, consider writing your own code, or simply giving your labels only non-zero values.
(255, 1)
255  labels  plus "unknown" 0:  {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 

True

In [120]:
def GetMin(values, size):

    assert(size>1)

    f = 0
    s = 0

    if (values[0] > values[1]):
        f = values[1]
        s = values[0]
    else :
        f = values[0]
        s = values[1]

    for i in range(2, size) :

        if values[i] < f :
            f = values[i]
        elif(values[i] < s):
            s = values[i]
        else :
            pass

    return f, s


def Modulate(cost_volume_arr):
    first = 0
    second = 0
    confidence = 0
    num_samples = cost_volume_arr.shape[0]

    for r in range(cost_volume_arr.shape[1]):
        for c in range(cost_volume_arr.shape[2]):

            values = cost_volume_arr[:, r, c]
            first, second = GetMin(values, num_samples)
            confidence = (second + 1) / (first + 1)
            cost_volume_arr[:, r, c] = values * confidence

    return cost_volume_arr

In [141]:
from scipy.special import softmax


def infer():
    
    unary = np.load('../src/0016_cost_volume_64_1.npy')
#     unary = Modulate(unary)
#     img = cv2.imread('../datasets/stone6_still/stone6_still_0001.png')
    img = cv2.imread('../datasets/0016_still/0016_still_0001.png')
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.pyrDown(img)
    img = cv2.pyrDown(img)

    cv2.imwrite('lab.png', img)

    depth_samples = []
    labels = unary.shape[0]
    min_depth=2
    max_depth=4
    num_samples = labels
    step = 1.0 / (num_samples - 1.0)

    # NewValue = (((OldValue - OldMin) * NewRange) / OldRange) + NewMin
    for val in range(num_samples):
        sample = (max_depth * min_depth) / (max_depth - (max_depth - min_depth) * val * step)
        sample = 1781.0/sample
        depth_samples.append(sample)
    
#     for idx, s in enumerate(depth_samples):
#         depth_samples[idx] = (((s - depth_samples[-1]) * 255) / (depth_samples[0] - depth_samples[-1]) )


    unary = unary.astype('float32')
    m = softmax(unary, axis=0)

    for i in range(unary.shape[1]):
        for j in range(unary.shape[2]):
            # print(np.argmin(unary[:,i,j]),":",np.min(unary[:,i,j]), np.argmax(unary[:,i,j]), ":", np.max(unary[:,i,j]))

            if np.sum(unary[:,i,j]) <= 1e-9:
                unary[:,i,j] = 0.0
            else:
                unary[:,i,j] = (unary[:,i,j]/np.sum(unary[:,i,j]))
    gd = np.argmin(unary, axis=0)

    gd_im = np.zeros((unary.shape[1], unary.shape[2]))
    for i in range(unary.shape[1]):
        for j in range(unary.shape[2]):

            gd_im[i,j] = depth_samples[gd[i,j]]
            # print(gd_im[i,j])

    # gd_im = ((( gd_im - 2) * 255) / 2)
    # print(np.max(gd_im))

    # plt.hist(gd_im, bins=np.arange(255))
    # plt.savefig('hist.png', )

    cv2.imwrite('unary_32.png', 255.0 - gd_im)
    sft = unary_from_softmax(unary)
    # print(unary[:,0,0], sft[:,0])
    # for i in range(unary.shape[1]):
    #     for j in range(unary.shape[2]):
    #         unary[:,i,j] /= np.sum(unary[:,i,j])
    unary = unary_from_softmax(unary)
    
    rgb = img
    d = dcrf.DenseCRF2D(rgb.shape[1], rgb.shape[0], labels)
    # get unary potentials (neg log probability)
    d.setUnaryEnergy(unary)
    # This adds the color-dependent term, i.e. features are (x,y,r,g,b).
    d.addPairwiseBilateral(sxy=(3, 3), srgb=(20, 20, 20), rgbim=rgb, compat=np.array([1, labels*0.11]), kernel=dcrf.DIAG_KERNEL, normalization=dcrf.NORMALIZE_SYMMETRIC)
    # d.addPairwiseBilateral(sxy=(80,80), srgb=(13,13,13), rgbim=rgb, compat=10, kernel=dcrf.DIAG_KERNEL, normalization=dcrf.NORMALIZE_SYMMETRIC)


    # Run inference steps.
    Q = d.inference(200)

    # Q, tmp1, tmp2 = d.startInference()
    # for i in range(50):
    #     print("KL-divergence at {}: {}".format(i, d.klDivergence(Q)))
    #     d.stepInference(Q, tmp1, tmp2)
    # Find out the most probable class for each pixel.
    # print(Q[])
    MAP = np.argmax(Q, axis=0).reshape((rgb.shape[:2]))
    depth_map = np.zeros((MAP.shape[0], MAP.shape[1]))
    for i in range(MAP.shape[0]):
        for j in range(MAP.shape[1]):
            
            sample = depth_samples[MAP[i,j]]
            sample = (((sample - depth_samples[-1]) * 255) / (depth_samples[0] - depth_samples[-1]) )
            depth_map[i,j] = sample

    cv2.imwrite('depth_map.png', depth_map)

    # unary = plane_sweep(min_depth=2, max_depth=4, scale=2, num_samples=32, patch_radius = 1)
    


In [142]:
infer()

Using trunc linear
